# Build a linear regression model on whoop data to predict next day's recovery based on previous day metrics

### target performance / metric: make recovery score predictions within 10% (lower or higher) than ground truth recovery score

165 days of data  
8 features  
1 target variable: recovery score  


In [7]:
#import statements
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [9]:
#load data
df = pd.read_csv("../data/feature_table.csv")
df.head()

,Unnamed: 0,Day Strain,Resting heart rate (bpm),Heart rate variability (ms),Average HR (bpm),Respiratory rate (rpm),Asleep duration (min),Deep (SWS) duration (min),REM duration (min),Recovery score %,Date
0,0,-1.260880,0.535925,-0.628835,1.148218,-1.692366,-2.215184,-0.598923,-0.746988,34.0,2025-07-16
1,1,0.362915,-0.492303,0.248181,-1.895251,-1.166490,-1.644456,0.071708,-0.648916,57.0,2025-07-16
2,2,-1.745170,1.906897,2.137139,-0.373517,0.411140,0.546404,-0.886336,-0.477288,84.0,2025-07-15
3,3,-1.232392,0.878668,0.113256,1.401840,-0.640613,0.693689,-0.455216,0.111149,59.0,2025-07-14
4,4,-0.064399,0.878668,-0.831223,0.133728,-1.955305,1.466933,-0.598923,1.435133,35.0,2025-07-13
